In [32]:
!pip install gradio transformers sentencepiece sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=9ddae2ae0c216c870d3c1bb664dc7917dfe914a2cfe26709399e6181760dade4
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [33]:
!git clone https://github.com/IDEA-CCNL/Fengshenbang-LM
import os
os.chdir('/content/Fengshenbang-LM/fengshen/')
!pwd

fatal: destination path 'Fengshenbang-LM' already exists and is not an empty directory.
/content/Fengshenbang-LM/fengshen


In [34]:
from models.deltalm.modeling_deltalm import DeltalmForConditionalGeneration
from transformers import AutoTokenizer

rdmodel = DeltalmForConditionalGeneration.from_pretrained("IDEA-CCNL/Randeng-Deltalm-362M-En-Zn")
rdtokenizer = AutoTokenizer.from_pretrained("microsoft/infoxlm-base")

def rdtrans(text):
  inputs = rdtokenizer(text, max_length=512, return_tensors="pt")
  generate_ids = rdmodel.generate(inputs["input_ids"], max_length=512)
  result = rdtokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  return result

In [35]:
import torch
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer,pipeline

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
dir = "Jyshen/Translation_en2zh"
mymodel = AutoModelForSeq2SeqLM.from_pretrained(dir)
mytokenizer = AutoTokenizer.from_pretrained(dir)

def mytranslate(inputs):
  trans = pipeline("translation_en_to_zh", model=mymodel, tokenizer=mytokenizer,
                       torch_dtype="float", device_map=True,device=DEVICE)
  return trans(inputs, max_length=450)[0]['translation_text']


In [36]:
import time
text = "When the five-member committee's investigation came to light Tuesday, Paxton suggested it was a political attack by the House's “liberal” Republican speaker, Dade Phelan. He called for Phelan’s resignation and accused him of being drunk during a marathon session last Friday. Phelan’s office brushed off the accusation as Paxton attempting to “save face.”"
print("RD_trans:")
s = time.time()
print(rdtrans(text))
print('Cost %d s'%(time.time()-s))
print('My_trans:')
s = time.time()
print(mytranslate(text))
print('Cost %d s'%(time.time()-s))

RD_trans:


Both `device` and `device_map` are specified. `device` will override `device_map`. You will most likely encounter unexpected behavior. Please remove `device` and keep `device_map`.


当五人委员会的调查在周二曝光时,帕克斯顿暗示这是众议院“自由派”共和党议长达德 ·菲兰的政治攻击。他呼吁菲兰辞职,并指责他在上周五的马拉松比赛中醉酒。菲兰的办公室驳回了对帕斯顿的指控。
Cost 256 s
My_trans:
当五名成员委员会的调查于周二亮相时,Paxton表示这是House的“自由党”共和党演讲者Dade Phelan的政治攻击。他呼吁Phelan辞职,并指控他在上周五的马拉松会议上喝酒。Phelan的办公室在Paxton试图“拯救面孔”时扫了指控。
Cost 4 s
